In [1]:
import numpy as np
import pandas as pd

In [ ]:
'''
22.12. 기준
인구소멸 위기지역 (기초자치단체(시,군) / 특례시(구):수원,고양,용인 행정구(구): 성남,안산,안양) top10
가임기 여성인구(20-39) / 60세이상 노령인구
'''

In [2]:
# 파일읽기
pe = pd.read_csv('data/주민등록인구집계현황.csv', encoding='euc-kr', thousands=',')
pe.head(3)

,연도,월,행정구역구분명,행정구역명,총 인구수,0~9세,10~19세,20~29세,30~39세,40~49세,...,10~19세 (여),20~29세 (여),30~39세 (여),40~49세 (여),50~59세 (여),60~69세 (여),70~79세 (여),80~89세 (여),90~99세 (여),100세 이상 (여)
0,2022,12,도,경기도,13589432,1040954,1327726,1737291,1882300,2284603,...,647283,821150,904073,1121889,1133428,878494,437251,252711,44184,1369
1,2022,12,시군,경기도 가평군,62197,2930,4463,5929,5397,7589,...,2186,2503,2433,3525,5090,6439,3653,2408,411,13
2,2022,12,읍면동,경기도 가평군 가평읍,19555,1020,1673,2094,1795,2630,...,778,937,848,1244,1583,1805,1054,739,126,6


In [3]:
pe.isna().sum().sum()

0

In [4]:
pe.columns

Index(['연도', '월', '행정구역구분명', '행정구역명', '총 인구수', '0~9세', '10~19세', '20~29세',
       '30~39세', '40~49세', '50~59세', '60~69세', '70~79세', '80~89세', '90~99세',
       '100세 이상', '총 인구수 (남)', '0~9세 (남)', '10~19세 (남)', '20~29세 (남)',
       '30~39세 (남)', '40~49세 (남)', '50~59세 (남)', '60~69세 (남)', '70~79세 (남)',
       '80~89세 (남)', '90~99세 (남)', '100세 이상 (남)', '총 인구수 (여)', '0~9세 (여)',
       '10~19세 (여)', '20~29세 (여)', '30~39세 (여)', '40~49세 (여)', '50~59세 (여)',
       '60~69세 (여)', '70~79세 (여)', '80~89세 (여)', '90~99세 (여)', '100세 이상 (여)'],
      dtype='object')

In [4]:
# 인구소멸위기지역 column만들기
pe['인구소멸위기지역'] = ((pe['20~29세 (여)']+pe['30~39세 (여)']) / (pe['60~69세']+pe['70~79세']+pe['80~89세']+pe['90~99세']+pe['100세 이상'])).round(2)

In [5]:
# dataframe 만들기 컬럼선택
pe = pe[['행정구역구분명','행정구역명','인구소멸위기지역']]
pe

,행정구역구분명,행정구역명,인구소멸위기지역
0,도,경기도,0.57
1,시군,경기도 가평군,0.20
2,읍면동,경기도 가평군 가평읍,0.26
3,읍면동,경기도 가평군 북면,0.09
4,읍면동,경기도 가평군 상면,0.11
...,...,...,...
615,읍면동,경기도 화성시 팔탄면,0.16
616,읍면동,경기도 화성시 향남읍,0.86
617,읍면동,경기도 화성시 화산동,0.47
618,읍면동,경기도 화성시동부출장소,NaN


In [ ]:
pe.isna().sum()

In [ ]:
pe[pe['행정구역구분명'].str.contains('읍면동')]

In [ ]:
# 읍면동 삭제
# 특정 문자포함 행 삭제
pe= pe.drop(pe[pe['행정구역구분명'] == '읍면동'].index)
pe.head()
# .index는 선택한 행들의 인덱스를 반환하는 속성

In [ ]:
pe.index

In [ ]:
pe['행정구역명'][231]

In [ ]:
# 방법2
# 원하는 행만 가져오기(filtering) / 도 빼기.
pe = pe[pe.행정구역구분명.str.contains('시군') | pe.행정구역구분명.str.contains('구')]

In [ ]:
# 특례시,행정구가 있는 도시(지방자치단체없는) 삭제
# .index는 선택한 행들의 인덱스를 반환하는 속성
# 방법1 이렇게는 하지말고
pe = pe.drop(pe[pe['행정구역명'] == '경기도 수원시 '].index)
pe = pe.drop(pe[pe['행정구역명'] == '경기도 고양시 '].index)
pe = pe.drop(pe[pe['행정구역명'] == '경기도 용인시 '].index)
pe = pe.drop(pe[pe['행정구역명'] == '경기도 성남시 '].index)
pe = pe.drop(pe[pe['행정구역명'] == '경기도 안산시 '].index)
pe = pe.drop(pe[pe['행정구역명'] == '경기도 안양시 '].index)

In [ ]:
# 방법2
# 수원시, 용인스 등 구가 있는 시는 제거
gu_city= ['수원시', '고양시', '용인시', '성남시', '안산시', '안양시']
pe.행정구역명 = pe.행정구역명.str.strip()
gu_pop = pe[pe.행정구역명,str.split()[-1].isin(gu_city)]

In [ ]:
# 방법2
# 수원시, 용인스 등 구가 있는 시는 제거 
# 인덱스 이름 반복문으로 가져오기
drop_index = []
for index in pe[pe.행정구역구분명 == '시군'].index:
    if pe.행정구역명[index].split(-1) in gu_city:
        drop_index.append(index)
drop_index

In [ ]:
# 방법2
pe.drop(drop_index, inplace=True)

In [ ]:
# 방법3
except_city = ['경기도 수원시', '경기도 고양시', '경기도 용인시', '경기도 성남시', '경기도 안산시', '경기도 안양시']

In [ ]:
# 방법3
pe[((pe.행정구역구분명 == '시군') | (pe.행정구역구분명 == '구')) & ~(pe['행정구역명'].str.strip().isin(except_city))]

In [ ]:
pe

In [ ]:
# 경기도 삭제
pe.drop([0], inplace=True) 
pe.head()

In [ ]:
pe.isna().sum()

In [ ]:
pe

In [ ]:
pe.to_csv('data/주민등록인구집계현황2.csv',index=False)

In [ ]:
# 0.5이하 인구소멸위기지역
# 인구소멸top10
sorted_pe = pe.sort_values(by='인구소멸위기지역')
sorted_pe.head(10)

In [ ]:
# 인구많은 top10
sorted_pe2 = sorted_pe.sort_values(by='인구소멸위기지역', ascending=False)
sorted_pe2.head(10)